# 30分钟搞懂它，加油！


## 导入数据

PyTorch 中有两个处理数据的工具，`torch.utils.data.DataLoader` 和 `torch.utils.data.Dataset`

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
# 从 datasets 下载开放数据集
training_data =  datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

我们将 `Dataset` 作为参数传递给 `DataLoader`，`DataLoader` 在我们的数据集上包装了一个可迭代的对象。并支持自动批处理（batche）、采样（sample）、混洗（shuffle）和多进程数据加载（multiprocess data loading）。这里我们定义 batch 为64，即数据加载器迭代中的每个元素将返回一批 64 个特征和标签的数据。

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


***

## 创建模型

创建了一个继承自 `nn.Module` 的类，在 PyTorch 定义神经网络。在 `__init__` 函数中定义网络层，并在函数中指定数据将如何通过 `forward` 中的网络。我们可以将神经网络移至 GPU，来加入模型的训练。

In [4]:
# 找到可以用于训练的 GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


--------------




## 优化模型参数
定义一个损失函数和一个优化器。

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单个训练循环中，模型对训练数据集进行预测（分批提供给它），并反向传播预测误差从而调整模型的参数。

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 计算预测误差
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

根据测试数据集检查模型的性能，以确保模型正在学习。

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

打印每一次迭代过程中的监控数据

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308198  [    0/60000]
loss: 2.303675  [ 6400/60000]
loss: 2.279196  [12800/60000]
loss: 2.273016  [19200/60000]
loss: 2.256729  [25600/60000]
loss: 2.224995  [32000/60000]
loss: 2.228598  [38400/60000]
loss: 2.191386  [44800/60000]
loss: 2.198501  [51200/60000]
loss: 2.156352  [57600/60000]
Test Error: 
 Accuracy: 48.6%, Avg loss: 2.157153 

Epoch 2
-------------------------------
loss: 2.171853  [    0/60000]
loss: 2.161133  [ 6400/60000]
loss: 2.100347  [12800/60000]
loss: 2.119125  [19200/60000]
loss: 2.059523  [25600/60000]
loss: 2.006031  [32000/60000]
loss: 2.032665  [38400/60000]
loss: 1.945392  [44800/60000]
loss: 1.962137  [51200/60000]
loss: 1.883276  [57600/60000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 1.881818 

Epoch 3
-------------------------------
loss: 1.920210  [    0/60000]
loss: 1.885050  [ 6400/60000]
loss: 1.766944  [12800/60000]
loss: 1.817860  [19200/60000]
loss: 1.691585  [25600/60000]
loss: 1.653540  [32000/600

--------------




## 保存模型

保存模型的常用方法是序列化内部状态字典（包含模型参数）。

In [9]:
torch.save(model.state_dict(), "model.pth")
print("模型已保存成功！")

模型已保存成功！


## 加载模型

加载模型的过程包括重新创建模型结构并将状态字典加载到其中。

In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

用加载的模型实现预测

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'预测结果是: "{predicted}", 实际标签是: "{actual}"')

预测结果是: "Ankle boot", 实际标签是: "Ankle boot"
